In [ ]:
# Eurostat: https://pypi.org/project/eurostat/

In [ ]:
import pandas as pd
import eurostat

In [ ]:
countries_list = ['EU27_2020', 'EA20', 'BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'EL', 'ES', 'FR', 'HR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL', 'AT', 'PL', 'PT', 'RO', 'SI', 'SK', 'FI', 'SE']

In [ ]:
def trimestre_a_fecha(trimestre):
    year, q = trimestre.split('-Q')
    month = {'1': '03', '2': '06', '3': '09', '4': '12'}[q]
    return f'{year}-{month}-01'

In [ ]:
nace_codes = pd.read_excel(open('Diccionario_etiquetas.xlsx', 'rb'), sheet_name='NACE') 
country_codes = pd.read_excel(open('Diccionario_etiquetas.xlsx', 'rb'), sheet_name='Countries') 

In [ ]:
# lfs -> sts_inpr_q (quarterly)

ipi_code = 'sts_inpr_q'

pars_ipi = eurostat.get_pars(ipi_code)

ipi_t_code = 'sts_inpr_q'

In [ ]:
for par in pars_ipi:
    par_values = eurostat.get_par_values(ipi_code, par)

    print(par, ": ", par_values)

In [ ]:
filters_ipi = {'startPeriod': '2018-01', 'geo': countries_list, 's_adj': 'SCA', 'unit': 'I21', 'nace_r2': ['B', 'C', 'D', 'F']}

filters_ipi_aux = {'startPeriod': '2018-01', 'geo': countries_list, 's_adj': 'SCA', 'unit': 'I21', 'nace_r2': ['B05', 'B06', 'B07', 'B08', 'B09', 'C10', 'C11', 
    'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'D35', 'E36']}

filters_ipi_t = {'startPeriod': '2018-01', 'geo': countries_list , 's_adj': 'SCA', 'unit': 'I21', 'nace_r2': ['B-D']}

In [ ]:
ipi_df = eurostat.get_data_df(ipi_code, filter_pars=filters_ipi)
ipi_df_aux = eurostat.get_data_df(ipi_code, filter_pars=filters_ipi_aux)
ipi_t_df = eurostat.get_data_df(ipi_t_code, filter_pars=filters_ipi_t)

ipi_df['Jerarquia_NACE'] = 'Seccion'
ipi_df_aux['Jerarquia_NACE'] = 'Division' 
ipi_t_df['Jerarquia_NACE'] = 'Total'

In [ ]:
ipi_all = pd.concat([ipi_df, ipi_df_aux, ipi_t_df])

ipi_all = ipi_all.drop(['freq', 'indic_bt', 'unit', 's_adj'], axis=1)

ipi_all['Indicador'] = 'IPI'

In [ ]:
ipi_all = ipi_all.merge(country_codes, on='geo\TIME_PERIOD', how='left')
ipi_all = ipi_all.drop('geo\TIME_PERIOD', axis=1)

In [ ]:
melted_ipi_df = ipi_all.melt(id_vars=['nace_r2', 'Descripcion', 'Jerarquia_NACE', 'Indicador'], var_name='Quarter', value_name='Value')

melted_ipi_df['Country'] = melted_ipi_df['Descripcion'].str.split('_').str[0]

pivot_ipi_df = melted_ipi_df.pivot(index=['nace_r2', 'Quarter', 'Jerarquia_NACE', 'Indicador'], columns='Country', values='Value').reset_index()

pivot_ipi_df.columns.name = None  

pivot_ipi_df.rename(columns={'Descripcion': 'periodo'}, inplace=True)

In [ ]:
pivot_ipi_final = pivot_ipi_df.merge(nace_codes, on='nace_r2', how='left')

In [ ]:
# Datasets a guardar

pivot_ipi_div = pivot_ipi_final[pivot_ipi_final.Jerarquia_NACE=="Division"]

pivot_ipi_sec = pivot_ipi_final[pivot_ipi_final.Jerarquia_NACE=="Seccion"]

pivot_ipi_t = pivot_ipi_final[pivot_ipi_final.Jerarquia_NACE=="Total"]

In [ ]:
with pd.ExcelWriter('IPI_pivot_q.xlsx') as writer:  

    pivot_ipi_t.to_excel(writer, sheet_name='IPI_total_pivot', index=False)

    pivot_ipi_sec.to_excel(writer, sheet_name='IPI_secciones_pivot', index=False)

    pivot_ipi_div.to_excel(writer, sheet_name='IPI_divisiones_pivot', index=False)

    pivot_ipi_final.to_excel(writer, sheet_name='IPI_panel_pivot', index=False)

In [ ]:
panel_ipi_df = pd.melt(ipi_all, 
    id_vars=['nace_r2', 'Descripcion', 'Jerarquia_NACE', 'Indicador'], 
    value_vars=[col for col in ipi_all.columns if 'Q' in col], 
    var_name='period', 
    value_name='value')

In [ ]:
panel_ipi_final = panel_ipi_df.merge(nace_codes, on='nace_r2', how='left')

In [ ]:
panel_ipi_final['fecha'] = panel_ipi_final['period'].apply(trimestre_a_fecha)
panel_ipi_final['fecha'] = pd.to_datetime(panel_ipi_final['fecha'], format='%Y-%m-%d')
panel_ipi_final['fecha'] = panel_ipi_final['fecha'].dt.strftime('%d/%m/%Y')

In [ ]:
# Datasets a guardar

panel_ipi_div = panel_ipi_final[panel_ipi_final.Jerarquia_NACE=="Division"]

panel_ipi_sec = panel_ipi_final[panel_ipi_final.Jerarquia_NACE=="Seccion"]

panel_ipi_t = panel_ipi_final[panel_ipi_final.Jerarquia_NACE=="Total"]

In [ ]:
with pd.ExcelWriter('IPI_q.xlsx') as writer:  

    panel_ipi_t.to_excel(writer, sheet_name='IPI_total', index=False)

    panel_ipi_sec.to_excel(writer, sheet_name='IPI_secciones', index=False)

    panel_ipi_div.to_excel(writer, sheet_name='IPI_divisiones', index=False)

    panel_ipi_final.to_excel(writer, sheet_name='IPI_panel', index=False)

In [ ]:
# IPI -> sts_inpr_m (monthly) 

ipi_code_m = 'sts_inpr_m'

pars_ipi_m = eurostat.get_pars(ipi_code_m)

ipi_t_code_m = 'sts_inpr_m'

In [ ]:
for par in pars_ipi_m:
    par_values = eurostat.get_par_values(ipi_code_m, par)

    print(par, ": ", par_values)

In [ ]:
filters_ipi_m = {'startPeriod': '2018-01', 'geo': countries_list, 's_adj': 'SCA', 'unit': 'I21', 'nace_r2': ['B', 'C', 'D']}

filters_ipi_aux_m = {'startPeriod': '2018-01', 'geo': countries_list, 's_adj': 'SCA', 'unit': 'I21', 'nace_r2': ['B05', 'B06', 'B07', 'B08', 'B09', 'C10', 'C11', 
    'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'D35', 'E36']}

filters_ipi_m_t = {'startPeriod': '2018-01', 'geo': countries_list, 's_adj': 'SCA', 'unit': 'I21', 'nace_r2': ['B-D']}

In [ ]:
ipi_df_m = eurostat.get_data_df(ipi_code_m, filter_pars=filters_ipi_m)
ipi_df_aux_m = eurostat.get_data_df(ipi_code_m, filter_pars=filters_ipi_aux_m)
ipi_t_df_m = eurostat.get_data_df(ipi_t_code_m, filter_pars=filters_ipi_m_t)

ipi_df_m['Jerarquia_NACE'] = 'Seccion'
ipi_df_aux_m['Jerarquia_NACE'] = 'Division' 
ipi_t_df_m['Jerarquia_NACE'] = 'Total'

In [ ]:
ipi_all_m = pd.concat([ipi_df_m, ipi_df_aux_m, ipi_t_df_m])

ipi_all_m = ipi_all_m.drop(['freq', 'indic_bt', 'unit', 's_adj'], axis=1)

ipi_all_m['Indicador'] = 'IPI'

In [ ]:
ipi_all_m = ipi_all_m.merge(country_codes, on='geo\TIME_PERIOD', how='left')
ipi_all_m = ipi_all_m.drop('geo\TIME_PERIOD', axis=1)

In [ ]:
melted_ipi_m_df = ipi_all_m.melt(id_vars=['nace_r2', 'Descripcion', 'Jerarquia_NACE', 'Indicador'], var_name='Quarter', value_name='Value')

melted_ipi_m_df['Country'] = melted_ipi_m_df['Descripcion'].str.split('_').str[0]

pivot_ipi_m_df = melted_ipi_m_df.pivot(index=['nace_r2', 'Quarter', 'Jerarquia_NACE', 'Indicador'], columns='Country', values='Value').reset_index()

pivot_ipi_m_df.columns.name = None  

pivot_ipi_m_df.rename(columns={'Descripcion': 'periodo'}, inplace=True)

In [ ]:
pivot_ipi_m_final = pivot_ipi_m_df.merge(nace_codes, on='nace_r2', how='left')

In [ ]:
# Datasets a guardar: Jerarquia NACE

pivot_ipi_m_div = pivot_ipi_m_final[pivot_ipi_m_final.Jerarquia_NACE=="Division"]

pivot_ipi_m_sec = pivot_ipi_m_final[pivot_ipi_m_final.Jerarquia_NACE=="Seccion"]

pivot_ipi_m_t = pivot_ipi_m_final[pivot_ipi_m_final.Jerarquia_NACE=="Total"]

In [ ]:
with pd.ExcelWriter('IPI_pivot_m.xlsx') as writer:  

    pivot_ipi_m_t.to_excel(writer, sheet_name='IPI_total_pivot_m', index=False)

    pivot_ipi_m_sec.to_excel(writer, sheet_name='IPI_secciones_pivot_m', index=False)

    pivot_ipi_m_div.to_excel(writer, sheet_name='IPI_divisiones_pivot_m', index=False)

    pivot_ipi_m_final.to_excel(writer, sheet_name='IPI_panel_pivot_m', index=False)

In [ ]:
# Datasets a guardar: Divisiones

pivot_ipi_m_div_B = pivot_ipi_m_div[pivot_ipi_m_div.division_code=="B"]

pivot_ipi_m_div_C = pivot_ipi_m_div[pivot_ipi_m_div.division_code=="C"]

pivot_ipi_m_div_D_E = pivot_ipi_m_div[pivot_ipi_m_div.division_code.isin(['D', 'E'])]

In [ ]:
with pd.ExcelWriter('IPI_pivot_m_divisiones.xlsx') as writer:  

    pivot_ipi_m_div_B.to_excel(writer, sheet_name='IPI_disiviones_B', index=False)

    pivot_ipi_m_div_C.to_excel(writer, sheet_name='IPI_disiviones_C', index=False)

    pivot_ipi_m_div_D_E.to_excel(writer, sheet_name='IPI_disiviones_DE', index=False)



In [ ]:
panel_ipi_df_m = pd.melt(ipi_all_m, 
    id_vars=['nace_r2', 'Descripcion', 'Jerarquia_NACE', 'Indicador'], 
    value_vars=[col for col in ipi_all_m.columns if '-' in col],
    var_name='period', 
    value_name='value')

In [ ]:
panel_ipi_final_m = panel_ipi_df_m.merge(nace_codes, on='nace_r2', how='left')

In [ ]:
panel_ipi_final_m['fecha'] = pd.to_datetime(panel_ipi_final_m['period'], format='%Y-%m')

panel_ipi_final_m['fecha'] = panel_ipi_final_m['fecha'].dt.strftime('%d/%m/%Y')

In [ ]:
# Datasets a guardar

panel_ipi_div_m = panel_ipi_final_m[panel_ipi_final_m.Jerarquia_NACE=="Division"]

panel_ipi_sec_m = panel_ipi_final_m[panel_ipi_final_m.Jerarquia_NACE=="Seccion"]

panel_ipi_t_m = panel_ipi_final_m[panel_ipi_final_m.Jerarquia_NACE=="Total"]

In [ ]:
with pd.ExcelWriter('IPI_m.xlsx') as writer:  

    panel_ipi_t_m.to_excel(writer, sheet_name='IPI_total_m', index=False)

    panel_ipi_sec_m.to_excel(writer, sheet_name='IPI_secciones_m', index=False)

    panel_ipi_div_m.to_excel(writer, sheet_name='IPI_divisiones_m', index=False)

    panel_ipi_final_m.to_excel(writer, sheet_name='IPI_panel_m', index=False)

In [ ]:
# Participacion laboral 

lfs_code = 'lfsq_egan2'

pars_lfs = eurostat.get_pars(lfs_code)

for par in pars_lfs:
    par_values = eurostat.get_par_values(lfs_code, par)
    print(par, ": ", par_values)

In [ ]:
lfs_code_aux = 'lfsq_egan22d'

pars_lfs_aux = eurostat.get_pars(lfs_code_aux)

for par in pars_lfs_aux:
    par_values = eurostat.get_par_values(lfs_code_aux, par)
    print(par, ": ", par_values)

In [ ]:
filters_lfs = {'startPeriod': '2018-Q1', 'age' : 'Y15-64','geo': countries_list, 'nace_r2': ['B', 'C', 'D']}

filters_lfs_aux = {'startPeriod': '2018-Q1', 'age' : 'Y15-64','geo': countries_list, 
    'nace_r2': ['B05', 'B06', 'B07', 'B08', 'B09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 
    'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'D35', 'E36']}

filters_lfs_t = {'startPeriod': '2018-Q1', 'age' : 'Y15-64','geo': countries_list, 'nace_r2': ['TOTAL']}

In [ ]:
lfs_df = eurostat.get_data_df(lfs_code, filter_pars=filters_lfs)
lfs_df_aux = eurostat.get_data_df(lfs_code_aux, filter_pars=filters_lfs_aux)
lfs_df_t = eurostat.get_data_df(lfs_code, filter_pars=filters_lfs_t)

lfs_df['Jerarquia_NACE'] = 'Division'
lfs_df_aux['Jerarquia_NACE'] = 'Seccion' 
lfs_df_t['Jerarquia_NACE'] = 'Total' 

In [ ]:
lfs_all = pd.concat([lfs_df, lfs_df_aux, lfs_df_t])

lfs_all = lfs_all.drop(['freq', 'unit', 'age'], axis=1)

lfs_all['Indicador'] = 'LFS'


In [ ]:
lfs_all= lfs_all.merge(country_codes, on='geo\TIME_PERIOD', how='left')
lfs_all = lfs_all.drop('geo\TIME_PERIOD', axis=1)

In [ ]:
lfs_all.head()

In [ ]:
melted_lfs_df = lfs_all.melt(id_vars=['nace_r2', 'sex', 'Descripcion', 'Jerarquia_NACE', 'Indicador'], var_name='Quarter', value_name='Value')

melted_lfs_df['Country'] = melted_lfs_df['Descripcion'].str.split('_').str[0]

pivot_lfs_df = melted_lfs_df.pivot(index=['nace_r2', 'sex', 'Quarter', 'Jerarquia_NACE', 'Indicador'], columns='Country', values='Value').reset_index()

pivot_lfs_df.columns.name = None  

pivot_lfs_df.rename(columns={'Descripcion': 'periodo'}, inplace=True)

In [ ]:
pivot_lfs_final = pivot_lfs_df.merge(nace_codes, on='nace_r2', how='left')

In [ ]:
# Datasets a guardar

pivot_lfs_div = pivot_lfs_final[pivot_lfs_final.Jerarquia_NACE=="Division"]

pivot_lfs_sec = pivot_lfs_final[pivot_lfs_final.Jerarquia_NACE=="Seccion"]

pivot_lfs_t = pivot_lfs_final[pivot_lfs_final.Jerarquia_NACE=="Total"]

In [ ]:
with pd.ExcelWriter('LFS_pivot_q.xlsx') as writer:  

    pivot_lfs_t.to_excel(writer, sheet_name='LFS_total_pivot', index=False)

    pivot_lfs_sec.to_excel(writer, sheet_name='LFS_secciones_pivot', index=False)

    pivot_lfs_div.to_excel(writer, sheet_name='LFS_divisiones_pivot', index=False)

    pivot_lfs_final.to_excel(writer, sheet_name='LFS_panel_pivot', index=False)

In [ ]:
panel_lfs_df = pd.melt(lfs_all, 
    id_vars=['nace_r2', 'Descripcion', 'sex', 'Jerarquia_NACE', 'Indicador'], 
    value_vars=[col for col in lfs_all.columns if 'Q' in col], 
    var_name='period', 
    value_name='value')

In [ ]:
panel_lfs_final = panel_lfs_df.merge(nace_codes, on='nace_r2', how='left')

In [ ]:
panel_lfs_final['fecha'] = panel_lfs_final['period'].apply(trimestre_a_fecha)
panel_lfs_final['fecha'] = pd.to_datetime(panel_lfs_final['fecha'], format='%Y-%m-%d')
panel_lfs_final['fecha'] = panel_lfs_final['fecha'].dt.strftime('%d/%m/%Y')

In [ ]:
def normalize_to_100(group):
    # base_value = group['value'].iloc[0]

    base_value = group[group['period'].str.startswith('2021')]['value'].mean()
    group['value'] = (group['value'] / base_value) * 100
    return group

In [ ]:
# Normalizacion a 100
panel_lfs_final = panel_lfs_final.groupby(['Descripcion', 'division_code', 'sex']).apply(normalize_to_100)

In [ ]:
# Datasets a guardar

panel_lfs_div = panel_lfs_final[panel_lfs_final.Jerarquia_NACE=="Division"]

panel_lfs_sec = panel_lfs_final[panel_lfs_final.Jerarquia_NACE=="Seccion"]

panel_lfs_t = panel_lfs_final[panel_lfs_final.Jerarquia_NACE=="Total"]

In [ ]:
with pd.ExcelWriter('LFS_q.xlsx') as writer:  

    panel_lfs_t.to_excel(writer, sheet_name='LFS_total', index=False)

    panel_lfs_sec.to_excel(writer, sheet_name='LFS_secciones', index=False)

    panel_lfs_div.to_excel(writer, sheet_name='LFS_divisiones', index=False)

    panel_lfs_final.to_excel(writer, sheet_name='LFS_pivot', index=False)

In [ ]:
# Union en un único dataset: Panel

panel_ipi_final['sex'] = pd.NA

In [ ]:
combined_dataset = pd.concat([panel_ipi_final, panel_lfs_final], ignore_index=True)

In [ ]:
combined_dataset['leyenda'] = combined_dataset['sex'].apply(
    lambda x: 'Empleados (Hombres)' if x == 'M' else 
    'Empleados (Mujeres)' if x == 'F' else 
    'Empleados (Total)' if x == 'T' else 
    'Indice de Producción Industrial'
)

In [ ]:
# Datasets a guardar

combined_dataset_div = combined_dataset[combined_dataset.Jerarquia_NACE=="Division"]

combined_dataset_sec = combined_dataset[combined_dataset.Jerarquia_NACE=="Seccion"]

combined_dataset_t = combined_dataset[combined_dataset.Jerarquia_NACE=="Total"]

In [ ]:
with pd.ExcelWriter('dataset_total.xlsx') as writer:  

    combined_dataset_t.to_excel(writer, sheet_name='total', index=False)

    combined_dataset_sec.to_excel(writer, sheet_name='secciones', index=False)

    combined_dataset_div.to_excel(writer, sheet_name='divisiones', index=False)

    combined_dataset.to_excel(writer, sheet_name='all', index=False)

In [ ]:
combined_dataset_2 = pd.concat([pivot_ipi_final, pivot_lfs_final], ignore_index=True)

In [ ]:
combined_dataset_2_div = combined_dataset_2[combined_dataset_2.Jerarquia_NACE=="Division"]

combined_dataset_2_sec = combined_dataset_2[combined_dataset_2.Jerarquia_NACE=="Seccion"]

combined_dataset_2_t = combined_dataset_2[combined_dataset_2.Jerarquia_NACE=="Total"]

In [ ]:
with pd.ExcelWriter('dataset_total_pivot.xlsx') as writer:  

    combined_dataset_t.to_excel(writer, sheet_name='total', index=False)

    combined_dataset_2_sec.to_excel(writer, sheet_name='secciones', index=False)

    combined_dataset_2_div.to_excel(writer, sheet_name='divisiones', index=False)

    combined_dataset_2.to_excel(writer, sheet_name='all', index=False)